In [35]:
!python --version

Python 3.10.15


In [36]:
!pip install -r requirements.txt

In [37]:
import os
import gradio as gr

from dotenv import load_dotenv
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.prompts.prompt import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM

In [38]:
load_dotenv('.env')

OLLAMA_HOST = os.getenv('OLLAMA_HOST')
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL')

In [39]:
llm = OllamaLLM(
    base_url=OLLAMA_HOST,
    model=OLLAMA_MODEL,
    verbose=True
)

In [40]:
md_directory = "data/"
md_files = [f for f in os.listdir(md_directory) if f.endswith('.md')]

print(f"Markdown files: {md_files}")

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=250,
    length_function=len,
    is_separator_regex=False,
)

all_pages = []
for md_file in md_files:
    md_path = os.path.join(md_directory, md_file)
    loader = UnstructuredMarkdownLoader(md_path)
    pages = loader.load_and_split(text_splitter)
    all_pages.extend(pages)

    print(f"Content of {md_file}:")
    for page in pages:
        print(page)

    all_pages.extend(pages)
    print(f"Loaded {len(pages)} pages from {md_file}")

print(f"Total number of pages loaded: {len(all_pages)}")

Markdown files: ['Лекция 1. Алгебра матриц.md']
Content of Лекция 1. Алгебра матриц.md:
page_content='\title{ Лекция 9. Алгебра матриц }

\section*{Матрица и ее частные виды}

Прямоугольной матрицей размера $m \times n$ называется совокупность $m \cdot n$ чисел, расположенных в виде прямоугольной таблицы, содержащей $m$ строк и $n$ столбцов.

Мы будем записывать матрицу в виде $$ A=\left(\begin{array}{cccc} a_{11} & a_{12} & \ldots & a_{1 n} \ a_{21} & a_{22} & \ldots & a_{2 n} \ \vdots & \vdots & : & : \ a_{m 1} & a_{m 2} & \ldots & a_{m n} \end{array}\right)=\left(a_{i j}\right)_{m \times n} $$

Числа $a_{i j}$ называются элементами матрицы, первый индекс матрицы (i) обозначает номер строки, второй $(j)$ - номер столбца. $$ \begin{aligned} & A=\left(\begin{array}{ccc} j & 1 & -3 \ 4 & -j & 5 \end{array}\right) \text {-комплексная матрица размера } 2 \times 3 . \ & B=\left(\begin{array}{ll} 1 & 2 \ 3 & 4 \end{array}\right) \text { - вещественная матрица размера } 2 \times 2 . \end{ali

In [41]:
embedding_model_nomic = OllamaEmbeddings(model="nomic-embed-text:latest")
db = Chroma.from_documents(all_pages, embedding_model_nomic)
retriever = db.as_retriever(search_kwargs={"k": 3})

In [42]:
template = """
You are a math teacher with artificial intelligence.
Your task is to create correct math assignments based on context.

### Context:
{context}

### Question:
{input}

### The answer:
"""

In [43]:
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [44]:
def get_response(query: str) -> str:
    """
    Функция, возвращающая ответ на вопрос, основанный на текстовых данных.

    :param query: текст вопроса
    :return: текст ответа
    """
    response = retrieval_chain.invoke({"input": query})
    return response["answer"]


In [45]:
iface = gr.Interface(
  fn=get_response,
  inputs=gr.Textbox(lines=2, placeholder="Введите свой промпт..."),
  outputs=gr.Markdown(),
  title="Qwen2_Math 1.5b",
  description="Напишите промпт на основе заданного контекста.",
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
